In [ ]:
# Fix the python path problem with jupyter
import sys
import os
PROJECT_BASE = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
assert PROJECT_BASE.split("/")[-1] == "sac"
sys.path.append(PROJECT_BASE)

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import itertools
from functools import reduce

import tensorflow as tf
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.special import logsumexp

from sac.distributions.real_nvp import RealNVP
from examples.real_nvp_2d_rl_example import RealNVP2dRlExample

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,8)

In [ ]:
SEED = np.random.randint(1, 10)
print("SEED: ", SEED)
np.random.seed(SEED)
tf.set_random_seed(SEED)

In [ ]:
MEANS = np.array([[0.2, 0.7], [-0.5, -0.5]])
VARIANCES = np.array([[0.05, 0.05], [0.1, 0.1]])
WEIGHTS = np.array([0.5, 1.0])

In [ ]:
tf.reset_default_graph()

%matplotlib notebook

subplots = fig, ax = plt.subplots(1, 1, figsize=(10, 10))

plt.ion()
ax.set_xlim((-2.0, 2.0))
ax.set_ylim((-2.0, 2.0))
ax.grid(True)

fig.canvas.draw()

example = RealNVP2dRlExample(subplots, WEIGHTS, MEANS, VARIANCES)

MIN, MAX = -2.1, 2.1
xs = np.linspace(MIN, MAX, 100)
ys = np.linspace(MIN, MAX, 100)
mesh_x, mesh_y = np.meshgrid(xs, ys)

x = np.stack((mesh_x.ravel(), mesh_y.ravel()), axis=1)
target_Q = example.session.run(
  example.policy_distribution.log_target,
  feed_dict={example.policy_distribution.z: x}
).reshape(mesh_x.shape)
p_z = np.exp(target_Q)

cs_target = plt.contour(mesh_x, mesh_y, p_z, 20, colors='r', linewidths=0.5)

example.run()